<a href="https://colab.research.google.com/github/omkardpatil/conformal_cooling_channels_design_optimization/blob/main/basic_QC_FE_solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import clear_output
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin
import fenics
from fenics import *
from mshr import *
from math import *
clear_output()

## Support functions

In [ ]:
def work(mesh, u, temp_threshold_for_fitness):
  # Define your mesh and function space
  V = FunctionSpace ( mesh, "Lagrange", 1 )

  # Create a function to interpolate 'u' to the function space
  u_interpolated = interpolate(u, V)

  # Get the values of 'u' at all nodes
  u_values = u_interpolated.vector().get_local()
  np_u = np.array(u_values)
  # Use np.ceil to round up each element to the nearest integer
  integer_array = np.ceil(np_u).astype(int)
  unique_elements, counts = np.unique(integer_array, return_counts=True)
  element_count_dict = dict(zip(unique_elements, counts))
  fitness=0
  for i, j in element_count_dict.items():
      r=i-temp_threshold_for_fitness
      if(r>0):
          fitness+=(j*j*r)
  return fitness

In [ ]:
def generate_mesh_here(circle, mesh_res):
  circle_x,circle_y,circle_r = circle
  domain = Circle(Point(0.0,0.0),1.0) -Rectangle(Point(-2.0,-2.0), Point(2.0,0.0))-Rectangle(Point(-2.0,0.0), Point(0.0,2.0))\
          - Circle(Point(circle_x,circle_y),circle_r)
  mesh = generate_mesh ( domain, mesh_res )
  return mesh

## Fenics solver

In [ ]:
from fenics import *
from mshr import *
from math import *
import os
import numpy as np
import time
import matplotlib.pyplot as plt
from matplotlib.colors import SymLogNorm



T_hot_edge = 150
T_cold_circle = 20
T_body_init = 40
point = (0.4,0.1)
threshold_dis = 0.02
temp_threshold_for_fitness=25
temp_threshold_for_stopping=30
mesh_res=20
t0=0.5 #time for which hot plastic will be in contact
#  Define time things
t_init = 0.0
t_final = 3.0
t_num = 300

u_origin=[]

def heat_implicit (circle, folder_name):
  circle_x,circle_y,circle_r = circle
  if not os.path.exists(folder_name):
    os.makedirs(folder_name)

  mesh  = generate_mesh_here(circle, mesh_res)
  plot ( mesh, title = 'heat_implicit Mesh' )

  filename =  folder_name+'heat_implicit_mesh.png'
  plt.savefig ( filename )
  plt.close ()

  V = FunctionSpace ( mesh, "Lagrange", 1 )

  rect_u = T_hot_edge #150.0
  def outer_circle_on ( x, on_boundary ):
    return 1-(x[0]**2+x[1]**2)<threshold_dis #x[0]+x[1]>1

  rect_bc = DirichletBC ( V, rect_u, outer_circle_on )

  circle_u = T_cold_circle#20.0
  def inner_circle_on ( x, on_boundary ):
    r = sqrt ( ( x[0] - circle_x ) ** 2 + ( x[1] - circle_y ) ** 2 )
    return abs(circle_r-r)< threshold_dis

  circle_bc = DirichletBC ( V, circle_u, inner_circle_on )
  bc = [ rect_bc, circle_bc ]
  bc_new = [circle_bc]

#  Define the trial functions (u) and test functions (v).
  u = TrialFunction ( V )
  v = TestFunction ( V )


#  The diffusivity is a constant.
  k = Constant ( 1.0 )
#  The source term is zero.
  f = Constant ( 0.0 )
  dt = ( t_final - t_init ) / t_num

#  Create U_INIT.
  u_init = Expression ( str(T_body_init), degree = 10 )
  u_old = interpolate ( u_init, V )
  fvt = ( u_old + dt * f ) * v * dx
  Auvt = u * v * dx + dt * dot ( grad(u), grad(v) ) * dx
  u = Function ( V )
  t = t_init

  v_min=20
  v_max=100
  for j in range ( 0, t_num + 1):
    if ( j % 2 == 0 ):
      label = 'Time = %g' % ( t )
      #########################################################################################################
      min_value = circle_u
      max_value = rect_u
      mappable = plot( u_old, title = label ,cmap='viridis', norm=SymLogNorm(linthresh=0.03, linscale=0.03, vmin=v_min, vmax=v_max))
      cbar = plt.colorbar(mappable)
      filename = folder_name+'heat_implicit_solution_%d.png' % ( j )
      plt.savefig ( filename )
      plt.close()
    t = t + dt
    if(t>t0):
      bc=bc_new
    solve ( Auvt == fvt, u, bc )
    u_old.assign ( u )
    max_value= u.vector().max()
    if(max_value<temp_threshold_for_stopping):
      fitness1 = work(mesh, u, temp_threshold_for_fitness)
      fitness2 = exp(t)
      return fitness1*fitness2
  fitness1 = work(mesh, u, temp_threshold_for_fitness)
  fitness2 = exp(t)
  return fitness1*fitness2

def heat_implicit_test (circle,folder_name):
  level = 30
  set_log_level ( level )
  return heat_implicit(circle,folder_name)

In [ ]:
folder_name='results1/'
circle_x = 0.4
circle_y = 0.4
circle_r = 0.1
circle = [circle_x,circle_y,circle_r]
Optimal_Solution =  [0.5703969581679545, 0.6, 0.1158924382356333]
circle =Optimal_Solution
ans =heat_implicit_test (circle,folder_name=folder_name)
clear_output()
print(ans)

791224.5700409901


In [ ]:
!zip -r /content/myfolder1.zip /content/results1
clear_output()